In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import joblib

# Charger le dataset depuis le fichier CSV
# Ce fichier contient les informations sur les offres et leurs caractéristiques
data = pd.read_csv("./data/deals_dataset.csv", delimiter=";", encoding="latin-1")

# Calculer le pourcentage de réduction
# Formule : (prix initial - prix actuel) / prix initial * 100
# Les valeurs manquantes sont remplacées par 0
data['discount_percentage'] = ((data['initial_price'] - data['price']) / data['initial_price'] * 100).fillna(0)

# Ajouter une colonne indiquant si la livraison est gratuite
# Si `shipping` est vide ou égal à 0, alors la livraison est gratuite
data['free_shipping'] = data['shipping'].fillna(0) == 0
data['free_shipping'] = data['free_shipping'].astype(int)  # Conversion en entier (1 pour vrai, 0 pour faux)

# Remplacer les valeurs manquantes dans `comments_count` par la médiane
data['comments_count'] = data['comments_count'].fillna(data['comments_count'].median())

# Créer la colonne cible `is_good_deal`
# Une offre est une "bonne affaire" si la température > 100 ou la réduction > 20%
data['is_good_deal'] = ((data['current_temperature'] > 100) | (data['discount_percentage'] > 20)).astype(int)

# Sélectionner les colonnes pertinentes pour l'entraînement du modèle
features = ['current_temperature', 'discount_percentage', 'free_shipping', 'comments_count']
X = data[features]  # Caractéristiques d'entrée
y = data['is_good_deal']  # Colonne cible

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialiser et entraîner un modèle Random Forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Sauvegarder le modèle entraîné et les colonnes utilisées
joblib.dump(rf_model, "random_forest_good_deal_model.pkl")
joblib.dump(features, "model_features.pkl")
print("Modèle et caractéristiques sauvegardés.")


Modèle et caractéristiques sauvegardés.


In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
import joblib

# Charger le modèle et les colonnes utilisées
rf_model = joblib.load("random_forest_good_deal_model.pkl")
features = joblib.load("model_features.pkl")

# Charger un nouveau dataset pour prédire si les offres sont des "bonnes affaires"
new_data = pd.read_csv("./data/deals_dataset.csv", delimiter=";", encoding="latin-1")

# Prétraitement des données (identique à celui utilisé lors de l'entraînement)
new_data['discount_percentage'] = ((new_data['initial_price'] - new_data['price']) / new_data['initial_price'] * 100).fillna(0)
new_data['free_shipping'] = new_data['shipping'].fillna(0) == 0
new_data['free_shipping'] = new_data['free_shipping'].astype(int)
new_data['comments_count'] = new_data['comments_count'].fillna(new_data['comments_count'].median())

# Sélectionner les colonnes pertinentes pour la prédiction
X_new = new_data[features]

# Faire les prédictions avec le modèle
new_data['is_good_deal'] = rf_model.predict(X_new)

# Sauvegarder les résultats dans un nouveau fichier CSV
new_data.to_csv("new_deals_with_predictions.csv", index=False)

# Afficher les résultats (track_id et prédiction)
print(new_data[['track_id', 'is_good_deal']])


            track_id  is_good_deal
0     thread_2914509             1
1     thread_2912979             1
2     thread_2911945             0
3     thread_2910548             0
4     thread_2909793             0
...              ...           ...
8221  thread_2886848             1
8222  thread_2886431             0
8223  thread_2875168             1
8224  thread_2875104             1
8225  thread_2868448             1

[8226 rows x 2 columns]


In [ ]:
# Importation des bibliothèques nécessaires
import pandas as pd
import joblib

# Charger le modèle et les caractéristiques
rf_model = joblib.load("random_forest_good_deal_model.pkl")
features = joblib.load("model_features.pkl")

print("Modèle et caractéristiques chargés avec succès.")
print("Caractéristiques utilisées :", features)

# Préparer un dataset fictif pour tester le modèle
new_data = pd.DataFrame({
    "current_temperature": [332, 689, -88],  # Température pour chaque offre
    "discount_percentage": [15.67, 25.0, 12.29],  # Réduction en pourcentage
    "free_shipping": [1, 0, 1],  # Indique si la livraison est gratuite
    "comments_count": [19, 82, 3]  # Nombre de commentaires pour chaque offre
})

print("Nouvelles données à tester :")
print(new_data)

# Faire des prédictions sur ces données
predictions = rf_model.predict(new_data[features])

# Ajouter les prédictions dans le dataframe
new_data['is_good_deal'] = predictions

print("Prédictions :")
print(new_data)


Modèle et caractéristiques chargés avec succès.
Caractéristiques utilisées : ['current_temperature', 'discount_percentage', 'free_shipping', 'comments_count']
